URL for scraping

`https://prosple.com/search-jobs?study_fields=502&keywords=Graduate&defaults_applied=1&locations=9692`

Location `9692` means Australia (whole)
Study field `502` means IT & Computer Science

# SELENIUM

In [1]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
# from webdriver_manager.firefox import GeckoDriverManager

In [2]:
gecko_path = "C:/Users/nguye/OneDrive/Desktop/ETL POST/driver/geckodriver/geckodriver.exe"
binary_path = 'C:/Program Files/Mozilla Firefox/firefox.exe'

options = webdriver.FirefoxOptions()
options.set_headless(True)
driver = webdriver.Firefox(options=options, executable_path=gecko_path, firefox_binary=binary_path)

C:\Users\nguye\AppData\Local\Temp\ipykernel_24292\920307596.py:5: DeprecationWarning: use setter for headless property instead of set_headless
  options.set_headless(True)


In [3]:
url = 'https://prosple.com/search-jobs?study_fields=502&keywords=Graduate&defaults_applied=1&locations=9692'
driver.get(url)
card_per_page = 20  # Define how many results come up per page
page_count = int(driver.find_elements_by_css_selector('a.bUjhYe')[-1].text)     # How many pages available

### Similar to above, let's build a template of extraction for the banner each time each of the cards are clicked

### **Let's look at all the information that I would need to extract, let's list them out:**

- 'Job title' CHECKED
Found at root <`driver.find_element_by_css_selector('h2.sc-eCssSg a')`> from banner

- 'Company Name' CHECKED
Found at root <`driver.find_element_by_css_selector('a.iSykQe')`> from banner

- 'Company Source' CHECKED
Found at root <`driver.find_element_by_css_selector('a.iSykQe').get_attribute('href')`> from banner

- 'Company Logo Source' CHECKED
Found at <`card.find_element_by_xpath('//img[@class="sc-dwcuIR jktQCG"]).get_attribute('src')`> for each card

- 'Source Link' CHECKED
Found at <`driver.find_element_by_css_selector('div.iOpAyz a').get_attribute('href')`> from banner

- 'Job Type' CHECKED
Found at <`driver.find_elements_by_css_selector('p.bprSzs')[0].text`> from banner

- 'Location' CBECKED
Found at <`driver.find_elements_by_css_selector('p.bprSzs')[1].text`> from banner

- 'Short description' CHECKED
Found at <`driver.find_element_by_css_selector('.oeIuV').text`> from banner

- 'Long description'

- 'Start date' CHECKED
Found at <`driver.find_elements_by_css_selector('div.krsExI div.eTEuVf')[0].text`>

- 'Open' and 'Close Date' CHECKED
Found at <`driver.find_elements_by_css_selector('div.krsExI div.eTEuVf')[1,2].text`>

- 'Vacancies' CHECKED
Found at <`driver.find_elements_by_css_selector('div.krsExI div.eTEuVf')[3].text`>

- 'Salary' CHECKED
Found at <`driver.find_elements_by_css_selector('div.krsExI div.eTEuVf')[4]`>

- 'Work Rights' CHECKED
Found at <`driver.find_elements_by_css_selector('div.gteQel div p')[0-..].text`> by looping

In [4]:
# Test area
# find_by_css_grace(query='h2.sc-eCssSg a').text
driver.find_elements_by_css_selector('div.krsExI div.eTEuVf')[4].get_attribute('innerHTML')

'<p class="SearchOpportunityContentstyle__SalaryDetail-sc-k2tet-40 ejxyuQ">AUD 55,000 - 65,000 <span class="SearchOpportunityContentstyle__SalaryRate-sc-k2tet-41 bPjXdZ">/ Year</span></p><p class="SearchOpportunityContentstyle__SalaryDescription-sc-k2tet-42 kMsVsG"></p>'

### Let's test the loop

In [5]:
def find_by_css_grace(query, itera=-1, looker='href'):
    if (itera == -1):
        try:
            return driver.find_element_by_css_selector(query).get_attribute(looker)
        except:
            return None
    elif (itera == -2):
        try:
            return driver.find_elements_by_css_selector(query)
        except:
            return None
    else:
        try:
            return driver.find_elements_by_css_selector(query)[itera].text
        except:
            return None
            

In [6]:
# import pandas as pd

def extract_info_iter(card):
    card.click()
    
    return {
        'title': find_by_css_grace(query='h2.sc-eCssSg a',looker='innerHTML'),
        'name' : find_by_css_grace(query='a.iSykQe', looker='innerHTML'),
        'logo' : card.find_element_by_css_selector('div div div img').get_attribute('src'),
        'source' : find_by_css_grace(query='div.iOpAyz a', looker='href'),
        'type' : find_by_css_grace(query='p.bprSzs',itera=0),
        'location' : find_by_css_grace(query='p.bprSzs',itera=1, looker='innerHTML'),
        'shr_desc' : find_by_css_grace(query='.oeIuV', looker='innerHTML'),
        'long_desc' : '\n\n'.join(element.text.strip() for element in find_by_css_grace(query='div.sc-kEjbxe.fBCLNp > *', itera=-2)),
        'str_date' : find_by_css_grace(query='div.krsExI div.eTEuVf',itera=0),
        'opn_date' : find_by_css_grace(query='div.krsExI div.eTEuVf',itera=1),
        'cls_date' : find_by_css_grace(query='div.krsExI div.eTEuVf',itera=2),
        'vac' : find_by_css_grace(query='div.krsExI div.eTEuVf',itera=3),
        'sal' : find_by_css_grace(query='div.krsExI div.eTEuVf',itera=4),
        'rights' : list(map(lambda x: x.get_attribute('innerHTML'), find_by_css_grace(query='div.gteQel div p', itera=-2)))
    }

### Let's start the loop for all pages

In [7]:
# text_elements = driver.find_elements_by_css_selector('div.sc-kEjbxe.fBCLNp > *')
# text_output = '\n\n'.join(element.text.strip() for element in text_elements)
# print(text_output)
driver.find_element_by_xpath('//a[@aria-label="Goto next page"]').get_attribute('href')

'https://prosple.com/search-jobs?study_fields=502&keywords=Graduate&defaults_applied=1&locations=9692&start=20'

In [8]:
# resulting list
result_list = []

for page in range(1,page_count+1):
    cards_on_page = driver.find_elements_by_class_name('sc-jifIRw')
    # extract info on the page
    result_list += list(map(extract_info_iter, cards_on_page))

    # try to get the next button, exit when no longer exist
    try:
        next_page_btn = driver.find_element_by_xpath('//a[@aria-label="Goto next page"]')

        # skip to next page
        driver.get(next_page_btn.get_attribute('href'))
    except:
        print('bruh')

    

bruh


In [9]:
import pandas as pd

df = pd.DataFrame(result_list)

In [10]:
df.shape

(435, 14)

In [11]:
driver.quit()

In [12]:
from datetime import datetime
df.to_csv(f'job_export/prosple_gradit_export_{datetime.now().strftime("%Y%m%d")}.csv')